<a href="https://colab.research.google.com/github/BlightG/Langchain/blob/master/Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.19
    Uninstalling langchain-core-0.3.19:
      Successfully uninstalled langchain-core-0.3.19
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.7
    Uninstalling langchain-0.3.7:
      Successfully uninstalled langchain-0.3.7


In [2]:
from dotenv import load_dotenv
from google.colab import userdata
load_dotenv()

False

In [5]:
from langchain.chains import LLMChain
from langchain_community.llms import HuggingFaceHub
from langchain.prompts import PromptTemplate

def main():

    huggingfacehub_api_token = userdata.get('HUGGINGFACE_TOKEN')
    sql_llm = HuggingFaceHub(
                repo_id="mrm8488/t5-base-finetuned-wikiSQL",
                huggingfacehub_api_token=huggingfacehub_api_token
               )

    eng_llm = HuggingFaceHub(
                repo_id="mrm8488/t5-base-finetuned-wikiSQL-sql-to-en",
                huggingfacehub_api_token=huggingfacehub_api_token
               )

    prompt = PromptTemplate(
                input_variables=["question"],
                template="Translate English to SQL: {question}"
                )

    sql_chain = prompt | sql_llm
    eng_chain = prompt | eng_llm

    print("Welcome to the SQL Query Generator CLI!")
    print("Type 'exit' to quit.\n")

    query_type = input("Do you want to generate SQL query or translate query to english? (sql/english): ").strip().lower()
    while query_type not in ["sql", "english"]:
        if query_type  == "exit":
            print("Goodbye!")
            return
        print("Invalid input. Please enter 'sql' or 'english'.")
        query_type = input("What query type would you like to generate? (sql/english): ").strip().lower()

    # CLI Loop
    while True:
        user_input = input("Enter your question: ").strip()
        if user_input.lower() == "exit":
            print("Goodbye!")
            return

        try:
            # Process the input through the chain
            if query_type == "sql":
                sql_query = sql_chain.invoke({"question": user_input})
                print(f"Generated SQL Query:\n{sql_query}\n")
            elif query_type == "english":
                eng_query = eng_chain.invoke({"question": user_input})
                print(f"Explanation for the SQL query:\n{eng_query}\n")

            switch = input("Do you want to switch query types? (yes/no): ").strip().lower()
            if switch == "yes":
                query_type = input("Do you want to generate SQL query or translate query to english? (sql/english): ").strip().lower()

                while query_type not in ["sql", "english"]:
                    if user_input.lower() == "exit":
                       print("Goodbye!")
                       return

                    print("Invalid input. Please enter 'sql' or 'english'.")
                    query_type = input("What query type would you like to generate? (sql/mongo): ").strip().lower()
            elif switch == "exit":
                print("Goodbye!")
                return

        except Exception as e:
            print(f"An error occurred: {e}")


if __name__ == "__main__":
    main()

<ipython-input-5-46c1bcdd34f4>:8: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  sql_llm = HuggingFaceHub(


Welcome to the SQL Query Generator CLI!
Type 'exit' to quit.

Do you want to generate SQL query or translate query to english? (sql/english): sql
Enter your question: what is the average age of users in the users table
Generated SQL Query:
SELECT AVG Age (in English) FROM table WHERE Table = users

Do you want to switch query types? (yes/no): yes
Do you want to generate SQL query or translate query to english? (sql/english): english
Enter your question: SELECT AVG Age (in English) FROM table WHERE Table = users
Explanation for the SQL query:
What is the average age of the user with a user's table?

Do you want to switch query types? (yes/no): exit
Goodbye!
